In [3]:
#Check whether required packages are inststalled
import subprocess
import sys

reqs = subprocess.check_output([sys.executable, '-m', 'pip', 'freeze'])
installed_packages = [r.decode().split('==')[0] for r in reqs.split()]
#print(installed_packages)

packages = ['pyodbc','pandas', 'requests', 'regex']

for package in packages:
    try:
        if package not in installed_packages:
            print(package + ' is not installed.')
            sys.exit()
    except:
        print('Error : Package does not exist. See Readme in Github for package installation')

# Import relevant modules
try:
    import pyodbc
    import os
    import requests
    from datetime import datetime
    import re
    
except:
    print('Error in importing modules.')

In [4]:
choose = 1
# Connect to the server
if choose == "1":
    # Contact admin for UID and PWD
    UID = input("Enter your user id : ")
    PWD = input("Enter password : ")

    #######Connection will be done using this conection string#######
    connection_string = "DefaultEndpointsProtocol=https;AccountName=metasqlstorage;AccountKey=VoSW7No7d0pMH29Cp2cbQrV4J4AoaZyMPCG5Ml\
    zmF+c0e6SvGlkvrbvdbuFe02Ee4OHedcoau4KD+AStDgvEIA==;EndpointSuffix=core.windows.net"

    ###### SQL connection part #
    conn = pyodbc.connect('Driver={SQL Server};'
                        'Server=meta-sql-resources.database.windows.net;'
                        'Database=metamaterial;'
                        'UID='+UID+';'
                        'PWD='+PWD+';'
                        'Trusted_Connection=no;'
                        'Encrypt=yes;')

    cursor = conn.cursor()
    print("Connected to the server \n")

    # Login Id used for data logging
    userid = input("Enter your login (name.surname) : ")
    login = userid + '@metamaterial.com'
    #########################################################

Connected to the server 



In [5]:
sql = "SELECT Name, Attachment from Subprocesses WHERE Attachment LIKE \'%jpg%\'"
cursor.execute(sql)
data = cursor.fetchone()
print(data)

('OI001', 'https://metasqlstorage.blob.core.windows.net/metamaterial/Subprocess-00684/500um.jpg')


In [ ]:
import cv2
import numpy as np
import pylab
import matplotlib.pyplot as plt
import pandas as pd
import requests
from io import BytesIO

# Kmeans 
def kmeans_color_quantization(image, clusters=2, rounds=1000):
    h, w = image.shape[:2]
    samples = np.zeros([h*w,3], dtype=np.float32)
    count = 0

    for x in range(h):
        for y in range(w):
            samples[count] = image[x][y]
            count += 1

    compactness, labels, centers = cv2.kmeans(samples,
            clusters, 
            None,
            (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 0.1), 
            rounds, 
            cv2.KMEANS_RANDOM_CENTERS)

    centers = np.uint8(centers)
    res = centers[labels.flatten()]
    return res.reshape((image.shape))

####################
'''
sql = "SELECT Name, Attachment from Subprocesses WHERE Attachment LIKE \'%jpg%\'"
cursor.execute(sql)
data = cursor.fetchall()
'''

#####################
# Load image
data = ['OI001', 'https://metasqlstorage.blob.core.windows.net/metamaterial/Subprocess-00684/500um.jpg']
file = data[0]
id = data[1]
filename = file[0:2]+'A'+file[2:]+'.jpg'
blob = BytesIO(requests.get(id).content)
img = cv2.imdecode(np.frombuffer(blob.read(), np.uint8), 1)

#cv2.imshow('a',img)
#cv2.waitKey(0)

#image = cv2.imread("C:\\Users\\LokeshwarBandhu\\Pictures\\PEN067\\PEN067-OI008-BF.jpg")
#original = image.copy()
original = img.copy()

# Perform kmeans color segmentation, grayscale, Otsu's threshold
kmeans = kmeans_color_quantization(img, clusters=2)
gray = cv2.cvtColor(kmeans, cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

# Find contours, remove tiny specs using contour area filtering, gather points
comet_data = []
#points_list = []
#points_size = []
#size_list = []
cnts, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2:]
AREA_THRESHOLD = 2
for c in cnts:
    area = cv2.contourArea(c)
    if area < AREA_THRESHOLD:
        cv2.drawContours(thresh, [c], -1, 0, -1)
    else:
        (x, y), radius = cv2.minEnclosingCircle(c)
        #points_list.append((int(x), int(y)))
        #points_size.append(radius)
        #size_list.append(area)
        comet_data.append([int(x), int(y), radius, area])

# Apply mask onto original image
result = cv2.bitwise_and(original, original, mask=thresh)
result[thresh==255] = (36,255,12)

# Overlay on original
original[thresh==255] = (36,255,12)

df=pd.DataFrame(comet_data, columns=['Location X', 'Location Y', 'radius (pixels)', 'area (contour)'])
print("Number of particles: {}".format(len(df)))
print("Average particle size: {:.3f}".format(df['area (contour)'].mean()))

filetxt = filename.split('.')[0]+'.csv'
df.to_csv(filetxt, index=None)

'''
# Display counted particles in image
# Display
cv2.imshow('kmeans', kmeans)
cv2.imshow('original', original)
cv2.imshow('thresh', thresh)
cv2.imshow('result', result)
#cv2.waitKey()

# Plot particle size distribution (histograms)
points_size_um = [i*5 for i in points_size] # assuming 1 pixel = 5 um
plt.hist(points_size_um, color = 'blue', edgecolor = 'black', bins = 100)
# Add labels
plt.title('Particle size distribution')
plt.xlabel('Particle size (um)')
plt.ylabel('No. of particles')
plt.show()
'''

In [8]:
file = data[0]
id = data[1]
filename = file[0:2]+'A'+file[2:]+'.jpg'

import requests

blob = requests.get(id).content
with open(filename,'wb') as f:
    f.write(blob)